In [2]:
from numba import cuda, vectorize, float32
from scipy import sparse
import random
import math
import string

import time
import numpy as np

In [ ]:
num_of_chars = 26*25

a = np.array([[random.randrange(0, 2) for i in range(num_of_chars)] for i in range(2000)])
b = np.array([[random.randrange(0, 2) for i in range(num_of_chars)] for i in range(400000)])

b_t = np.transpose(b)

start_time = time.time()
c_true = np.matmul(a, b_t)
end_time = time.time()

end_time-start_time

In [ ]:
c = np.zeros(shape=(a.shape[0], b.shape[0]))

In [ ]:
c.shape

In [193]:
#a = sparse.csr_matrix(a)
#b_t = sparse.csr_matrix(b_t)

In [194]:
@cuda.jit
def pairwise_distance(A, B, C):
    """
    Perform pairwise distance calculation
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += abs(A[row, k] - B[k, col])
        C[row, col] = tmp

In [197]:
TPB = 128*5 #<---threads per block

blockspergrid_x = int(math.ceil(a.shape[0] / TPB))
blockspergrid_y = int(math.ceil(b.shape[0] / TPB))

blockspergrid = (blockspergrid_x, blockspergrid_y)

TPB, blockspergrid

(640, (4, 63))

In [198]:
start_time  =time.time()
pairwise_distance[[TPB, TPB], blockspergrid](a, b_t, c)
end_time  =time.time()

In [199]:
end_time-start_time

2.0437729358673096

In [200]:
c

array([[111., 101., 104., ..., 110.,  96.,  94.],
       [108., 100., 111., ..., 111., 105., 103.],
       [ 95., 105., 102., ...,  98., 104., 110.],
       ...,
       [106., 100., 111., ..., 103.,  95., 103.],
       [110., 102.,  99., ...,  91., 101., 101.],
       [109.,  99., 104., ..., 110., 102., 100.]])

In [202]:
sum(abs((a[0] - b)[0]))

111

In [179]:
a

array([[0, 1, 1, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 0, ..., 0, 1, 1],
       [1, 0, 0, ..., 1, 1, 1],
       [0, 1, 0, ..., 0, 0, 1]])

In [180]:
b

array([[1, 0, 1, ..., 1, 0, 1],
       [0, 1, 0, ..., 0, 1, 0],
       [1, 0, 1, ..., 0, 1, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])